In [3]:
import pandas as pd
import os
import csv

# Location of Traffic Flow
x1 = 0
x2 = 2
lane_length = x2 - x1

# Time Period
time_span = 1500 # seconds
time_interval = 15 # seconds

data_path = "D:/荷兰/TUe/课程/Team Project/Dataset/highd-dataset-v1.0/data"
n_files = 60
data_dict = {}

# Map ID List
map_ids = [1, 2, 3, 4, 5, 6]

output_path = "output"

if not os.path.exists(output_path):
    os.makedirs(output_path)

for map_id in map_ids:
    print(f"processing map {map_id}...\r", end="")
    
    for i in range(n_files):
        file_name1 = f"{i+1:02d}_tracks.csv"
        file_path1 = os.path.join(data_path, file_name1)
        file_name2 = f"{i+1:02d}_recordingMeta.csv"
        file_path2 = os.path.join(data_path, file_name2)
        recording_meta = pd.read_csv(file_path2)

        if recording_meta["locationId"][0] != map_id:
            continue

        tracks = pd.read_csv(file_path1)
        file_base_name = os.path.splitext(file_name1)[0]
        
        counts = []
        densities = []

        for t in range(0, time_span, time_interval):
            # Calculating Forward Traffic Flow
            mask = (tracks["frame"] >= t * recording_meta["frameRate"][0]) & (tracks["frame"] < (t + time_interval) * recording_meta["frameRate"][0]) & (tracks["x"] >= x1) & (tracks["x"] <= x2)
            vehicles = tracks[mask & (tracks["y"] >= 0)]["id"].unique()
            count = len(vehicles)
            density = count / (time_interval / 60) / lane_length
            counts.append(count)
            densities.append(density)

        # Write traffic flow intensities to a CSV file
        output_file_name = f"map{map_id}_file{i+1:02d}_density.csv"
        output_file_path = os.path.join(output_path, output_file_name)
        with open(output_file_path, mode='w', newline='') as output_file:
            output_writer = csv.writer(output_file)
            output_writer.writerow(["Time (seconds)", "Vehicle count", "Density (vehicles per meter)"])
            for t, (count, density) in enumerate(zip(counts, densities)):
                output_writer.writerow([t * time_interval, count, density])
                
print("Done!")



Done!ssing map 6...
